<h1> <b> Scripts for Location Data from Source </b> </h1>
<i> Source: <a> https://www.google.com/maps</a> </i>

In [52]:
from selenium.webdriver import Chrome
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import pandas as pd

<br>
<h1> <b> Functions Used </b> </h1> 

In [77]:
def search(query, driver):
    search_box = driver.find_element(By.ID, 'searchboxinput')
    search_box.send_keys(query)
    search_box.send_keys(Keys.ENTER)
    
    sleep(2)

def init_base():
    path = 'chromedriver\chromedriver.exe'
    driver = Chrome(path)
       
    return driver

def add_data(transport_data_temp, transportation_data):
    transportation_data['Bus Stations'].extend(transport_data_temp['Bus Stations'])
    transportation_data['Train Stations'].extend(transport_data_temp['Train Stations'])
    transportation_data['Airports'].extend(transport_data_temp['Airports'])
    transportation_data['Ports'].extend(transport_data_temp['Ports'])

def search_by_category(driver, transport_data_temp, province):
    '''
    search the data via categories; given by the keys in 
    transport data temp
    '''
    for category in list(transport_data_temp.keys()): # where category is a string; the key for all transportation categories in transport_data_temp
        category_query = category + ' near ' + province
        search(category_query, driver)
        results = driver.find_elements(By.CSS_SELECTOR, ('.mapsConsumerUiSubviewSectionGm2Placeresultcontainer__result-container'
                                                        + ' a'))
        if len(results) == 0:
            result = driver.find_element(By.CSS_SELECTOR, '.section-hero-header-title-title span').text
            results_formatted = [result]
        else:
            results_formatted = [result.get_attribute('aria-label') for result in results]
            
        transport_data_temp[category].append(results_formatted)
        
        x_button = driver.find_element(By.CSS_SELECTOR, '.gsst_a #sb_cb50' )
        x_button.click()
    
def search_by_province(driver, province_batch, transportation_data):
    '''
    this function searches all transportations as per category for each
    province in the batch, then adds only when all provinces have been 
    searched.
    '''
    transportation_data_temp = {
        'Bus Stations':[],
        'Train Stations':[],
        'Airports':[],
        'Ports':[]
    }
    
    for province in province_batch:
        search_by_category(driver, transportation_data_temp, province)
    
    # once all provinces have been searched, add via extend all the searched data
    add_data(transportation_data_temp, transportation_data)    
    

        
        


<br>
<h1> <b> Data Extraction and Cleaning </b> </h1>

In [93]:
location_data = pd.read_csv('data/Province_Data.csv')

In [94]:
location_data.head()

Province  Population Main Language
0   Metro Manila    12877253      Filipino
1  Camarines Sur     1952544         Bikol
2        Cagayan      675950       Ilocano
3   Ilocos Norte      593081       Ilocano
4     Ilocos Sur      689668       Ilocano

In [95]:
location_data

Province  Population Main Language
0       Metro Manila    12877253      Filipino
1      Camarines Sur     1952544         Bikol
2            Cagayan      675950       Ilocano
3       Ilocos Norte      593081       Ilocano
4         Ilocos Sur      689668       Ilocano
5               Cebu    76500000       Cebuano
6           Pampanga     2198110   Kapampangan
7            Palawan      849469      Palawano
8             Ifugao      202802         Batad
9              Davao      632588      Davaoeño
10             Aklan      574823       Aklanon
11    Southern Leyte      421750       Cebuano
12           Benguet      446224       Ilocano
13             Bohol     1313560    Binol-anon
14             Albay     1314826         Bicol
15           Talisay      227645       Cebuano
16          Zambales      590848       Tagalog
17             Rizal     2884227       Tagalog
18             Pasay      416522      Filipino
19  Misamis Oriental      888509       Cebuano
20            Laguna     3035081       Tagalog

In [96]:
# format population column -> change to thousands
location_data['Population (in thousands)'] = location_data['Population'].apply(lambda x: x/1000)

# add secondary language data -> all filipino
location_data.rename(columns={'Main':'Primary Language'}, inplace=True)
location_data['Secondary Language'] = 'Filipino'

In [97]:
# get lenght of batches
location_data.shape

# do search by batches of 7 provinces 

(21, 5)

In [67]:
# code for adding transportation data
driver = init_base()
driver.get('https://www.google.com/maps')

In [81]:
transportation_data = {
    'Bus Stations':[],
    'Train Stations':[],
    'Airports':[],
    'Ports':[]
}

provinces = location_data.Province.to_list()

In [82]:
# batch 1
i = 0
province_batch = search_by_province(driver, provinces[i:i+7], transportation_data)
i+=7

In [83]:
# batch 2
province_batch = search_by_province(driver, provinces[i:i+7], transportation_data)
i+=7

In [84]:
# batch 3
province_batch = search_by_province(driver, provinces[i:i+7], transportation_data)
i+=7

In [98]:
location_data['Bus Stations'] = transportation_data['Bus Stations']
location_data['Train Stations'] = transportation_data['Train Stations']
location_data['Airports'] = transportation_data['Airports']
location_data['Ports'] = transportation_data['Ports']

In [100]:
# remove population in millions
location_data.drop('Population', inplace=True, axis=1)

# format the location data
location_data_formatted = location_data[['Population (in thousands)','Main Language', 'Secondary Language', 'Bus Stations', 'Train Stations', 'Airports', 'Ports']]

In [101]:
location_data_formatted

Population (in thousands) Main Language Secondary Language  \
0                   12877.253      Filipino           Filipino   
1                    1952.544         Bikol           Filipino   
2                     675.950       Ilocano           Filipino   
3                     593.081       Ilocano           Filipino   
4                     689.668       Ilocano           Filipino   
5                   76500.000       Cebuano           Filipino   
6                    2198.110   Kapampangan           Filipino   
7                     849.469      Palawano           Filipino   
8                     202.802         Batad           Filipino   
9                     632.588      Davaoeño           Filipino   
10                    574.823       Aklanon           Filipino   
11                    421.750       Cebuano           Filipino   
12                    446.224       Ilocano           Filipino   
13                   1313.560    Binol-anon           Filipino   
14                   1314.826         Bicol           Filipino   
15                    227.645       Cebuano           Filipino   
16                    590.848       Tagalog           Filipino   
17                   2884.227       Tagalog           Filipino   
18                    416.522      Filipino           Filipino   
19                    888.509       Cebuano           Filipino   
20                   3035.081       Tagalog           Filipino   

                                         Bus Stations  \
0   [UERM Memorial Medical Center, J. Ruiz, Aurora...   
1   [Ocampo, Ocampo, Baao, Baao, Pili, Pili, Iriga...   
2   [Camalaniugan, Camalaniugan, Lal-Lo, Lal-Lo, A...   
3   [San Nicolas, San Nicolas, Laoag, Laoag, Parta...   
4   [Candon City, Candon City, Candon City, Candon...   
5   [Professional Regulations Commission, Securiti...   
6   [San Fernando City, San Fernando City, Transpo...   
7   [San Jose Terminal, Roro Bus Station, Roxas Bu...   
8   [Lagawe, Lagawe, Lagawa, Kiangan, Banaue, Bana...   
9   [Calinan, Calinan Public Market Jeepney Stop, ...   
10  [Caticlan, Caticlan, Kalibo, Kalibo, Bangan, B...   
11  [Libagon, Libagon, Liloan, Liloan, Mahaplag, M...   
12  [Baguio City, Victory Liner Bus Terminal, Bagu...   
13  [Cordova, Senor San Roque Parish Church, Senor...   
14  [Legazpi City, Legazpi City, Daraga, Daraga, C...   
15  [Bulacao, Inayawan Church, Rizal Ave, Rizal Av...   
16  [Masinloc, Masinloc, Palauig, Palauig, Palauig...   
17  [Olalia, Robinsons Place Antipolo, Senora Dela...   
18  [EDSA-Taft LRT/MRT, EDSA-Taft LRT/MRT, Kabayan...   
19  [Cagayan De Oro City, Cagayan De Oro City, Pob...   
20  [Coast-Ocean, Coast-Laguna, Laguna Beach Bus S...   

                                       Train Stations  \
0   [J. Ruiz, V. Mapa, Gilmore, Santa Mesa, Pureza...   
1   [Sipocot, PNR Baao Station, Train Station, PNR...   
2   [PNR Tuguegarao Central Station, GV Florida Bu...   
3   [GV Florida Bus Station, Caltex Airport Laoag,...   
4   [Vigan Terminal, Candon City, Aniceto Bus Line...   
5                           [Cebu city train station]   
6   [Gov.Pascual, Sangandaan, Asistio, Monumento, ...   
7   [El Nido Transport Terminal, Coron Bus and Jee...   
8   [Rapid Kl Train Station 2, Hanging Bridge, Rap...   
9   [Catalunan Grande Jeepney Station, Davao City ...   
10  [West Aklan Transport Terminal, Ceres Liner, K...   
11  [Silago Bus Terminal, PETRON GAS STATION - MAA...   
12  [Baguio City, Sunshine Bus Terminal, Victory L...   
13  [Sikatuna Bus Terminal, Berg Station Garden Tr...   
14  [Legazpi, Daraga, Train station, Legazpi Grand...   
15  [Cebu city train station, Litoda Tricycle Term...   
16  [Sta. Cruz, Zambales, Tarlac City Philippine N...   
17  [Santolan MRT-2 Station, Katipunan, Anonas, Sa...   
18  [Taft Avenue, EDSA, Baclaran, Libertad, EDSA, ...   
19  [Buena Oro Jeepney Terminal, Agora Bus Termina...   
20  [Laguna Niguel/ Mission Viejo Metrolink Statio...   

                                         

In [103]:
file_path = 'data/Location_Source.csv'
location_data_formatted.to_csv(file_path, index=False)

In [106]:
# check
df = pd.read_csv('data/Location_Source.csv')
df.head()

Population (in thousands) Main Language Secondary Language  \
0                  12877.253      Filipino           Filipino   
1                   1952.544         Bikol           Filipino   
2                    675.950       Ilocano           Filipino   
3                    593.081       Ilocano           Filipino   
4                    689.668       Ilocano           Filipino   

                                        Bus Stations  \
0  ['UERM Memorial Medical Center', 'J. Ruiz', 'A...   
1  ['Ocampo', 'Ocampo', 'Baao', 'Baao', 'Pili', '...   
2  ['Camalaniugan', 'Camalaniugan', 'Lal-Lo', 'La...   
3  ['San Nicolas', 'San Nicolas', 'Laoag', 'Laoag...   
4  ['Candon City', 'Candon City', 'Candon City', ...   

                                      Train Stations  \
0  ['J. Ruiz', 'V. Mapa', 'Gilmore', 'Santa Mesa'...   
1  ['Sipocot', 'PNR Baao Station', 'Train Station...   
2  ['PNR Tuguegarao Central Station', 'GV Florida...   
3  ['GV Florida Bus Station', 'Caltex Airport Lao...   
4  ['Vigan Terminal', 'Candon City', 'Aniceto Bus...   

                                            Airports  \
0  ['Ninoy Aquino International Airport', 'Manila...   
1  ['Naga Airport', 'Bicol International Airport'...   
2  ['Cagayan North International Airport', 'Tugue...   
3  ['Laoag International Airport', 'Liagens', 'La...   
4  ['Vigan Airport', 'Laoag City International Ai...   

                                               Ports  
0                                 ['Port of Manila']  
1  ['Sabang Port', 'Port of Pasacao', 'Guijalo Po...  
2  ['San Vicente Fish Port', 'Port Irene', 'Casam...  
3  ['Currimao Port', 'Currimao', 'PAGSANAHAN PORT...  
4  ['Salomague Port', 'Quirino Tagudin ilocos sur...